# Pet Database
#### Group 11
### D532 Final Project

## imports:
####      1.SQLite3 this is our database
####      2.pandas - this allows us to manipulate the data and run sql

In [36]:
import sqlite3
import pandas as pd

### Connecting to the pets sqlite3 database

In [37]:
conn = sqlite3.connect('pets.db')
c = conn.cursor()

### Importing Raw CSV of our data

In [38]:
read_emp = pd.read_csv(r'Adoptable_Pets.csv')
read_emp.to_sql('pets_raw_csv', conn, if_exists='replace', index = False)

58

### Intake Table

In [39]:
c.execute('drop table if exists intake;')
intake_table = '''
    CREATE TABLE IF NOT EXISTS intake(
intake_type_id varchar(20) PRIMARY KEY,
intake_type_desc varchar(50) NOT NULL,
CONSTRAINT CHK_INTAKE CHECK (intake_type_id IN ('OWNER SUR', 'EUTH REQ', 'STRAY', 'FOSTER', 'BOARDING', 'RETURN'))
);'''
c.execute(intake_table)

### Adoption Status

In [40]:
c.execute('DROP TABLE IF EXISTS adoption_status;')
adoption_status_table = '''
    CREATE TABLE IF NOT EXISTS adoption_status(
        animal_id INTEGER PRIMARY KEY,
        adoption_status BOOLEAN,
        CONSTRAINT CHK_ADOPT CHECK (adoption_status IN (0, 1))
    );'''
c.execute(adoption_status_table)


### Pet Size

In [41]:
pet_size_table = '''
    CREATE TABLE IF NOT EXISTS pet_size(
pet_size_id char(1) PRIMARY KEY,
pet_size_desc varchar(10),
CONSTRAINT CHK_PETSIZE_ID CHECK (pet_size_id IN ('S', 'M', 'L')),
CONSTRAINT CHK_PETSIZE CHECK (pet_size_desc IN ('SMALL','MED','LARGE'))
);'''
c.execute(pet_size_table)

### Sex

In [42]:
sex_table = '''
    CREATE TABLE IF NOT EXISTS sex_type(
    sex_id CHAR(1) PRIMARY KEY,
    sex_desc VARCHAR(10) NOT NULL,
    CONSTRAINT CHK_sex_id CHECK (sex_id IN ('M', 'F', 'U', 'N', 'S')),
    CONSTRAINT CHK_sex CHECK (sex_desc IN ('Male', 'Female', 'Unidentified', 'Neutral', 'Spayed'))
);'''
c.execute(sex_table)

# Constraints

##### Foreign keys:

- (intake_type_id) - intake(intake_id)
- (pet_size_id) - pet_size(pet_size_id)
- (sex_id) - sex(sex_id)
- Primary key - animal_id

In [43]:
# Drop the 'animal' table if it exists
c.execute('DROP TABLE IF EXISTS animal')

# Create the 'animal' table
animal_table = '''
    CREATE TABLE animal (
        animal_id INTEGER PRIMARY KEY AUTOINCREMENT,
        animal_type VARCHAR(100) NOT NULL,
        pet_name VARCHAR(200) NOT NULL,
        intake_type VARCHAR(20) NOT NULL,
        in_date DATE NOT NULL,
        pet_age VARCHAR(20) NOT NULL,
        pet_size VARCHAR(10) NOT NULL,
        color VARCHAR(50) NOT NULL,
        breed VARCHAR(100) NOT NULL,
        sex CHAR(1) NOT NULL,
        url_link VARCHAR(400),
        crossing VARCHAR(400),
        FOREIGN KEY (intake_type) REFERENCES intake(intake_type_id),
        FOREIGN KEY (pet_size) REFERENCES pet_size(pet_size_id),
        FOREIGN KEY (sex) REFERENCES sex_type(sex_id)
    )
'''
c.execute(animal_table)

In [44]:
# Load data from pets_raw_csv into a DataFrame
df = pd.read_sql_query('SELECT * FROM pets_raw_csv', conn)

# Convert the 'in_date' column to the desired format
df['in_date'] = pd.to_datetime(df['in_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

# Insert the modified DataFrame into the 'animal' table
df.to_sql('animal', conn, if_exists='append', index=False)



58

In [45]:
c.execute('select * FROM animal;')
c.fetchall()

[(1,
  'OTHER',
  'OSCAR',
  'OWNER SUR',
  '2024-01-28',
  '6 YEARS',
  'SMALL',
  'BROWN',
  'TURTLE',
  'M',
  'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A451630&location=MONT',
  None),
 (2,
  'OTHER',
  'FRANKLIN',
  'OWNER SUR',
  '2024-01-28',
  '10 YEARS',
  'MED',
  'RED',
  'TURTLE / REDEARED SLIDER',
  'M',
  'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A439979&location=MONT',
  None),
 (3,
  'DOG',
  '*RITA',
  'OWNER SUR',
  '2024-02-05',
  '2 YEARS',
  'MED',
  'BRINDLE',
  'AM PIT BULL TER',
  'S',
  'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A507522&location=MONT',
  None),
 (4,
  'OTHER',
  'YERTLE',
  'OWNER SUR',
  '2024-01-28',
  '7 YEARS',
  'SMALL',
  'GREEN',
  'TURTLE',
  'F',
  'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A444157&location=MONT',
  None),
 (5,
  'DOG',
  'NOVA',
  'EUTH REQ',
  '2023-12-08',
  '2 YEARS',
  'MED',
  'TAN / WHITE',
  'AM PIT BULL TER / MIX',
  'S',
  'http://www.petharbor.com/get_image.asp?r

##Attribute Inserts

In [46]:
pet_size_insert = '''INSERT INTO pet_size(pet_size_id, pet_size_desc) VALUES
('S', 'SMALL'),
('M', 'MED'),
('L', 'LARGE')'''
c.execute(pet_size_insert)


In [47]:
animal_update = '''UPDATE animal
SET pet_size = CASE
    WHEN pet_size = 'SMALL' THEN 'S'
    WHEN pet_size = 'MED' THEN 'M'
    WHEN pet_size = 'LARGE' THEN 'L'
    ELSE pet_size
END;'''
c.execute(animal_update)

In [48]:
intake_type_insert = '''
INSERT INTO intake (intake_type_id, intake_type_desc) VALUES
      ('OWNER SUR', 'Owner Surrender'),
      ('EUTH REQ', 'Euthanasia Request'),
      ('STRAY' ,'Stray'),
      ('FOSTER', 'Foster'),
      ('BOARDING' ,'Boarding'),
      ('RETURN' ,'Return');'''
c.execute(intake_type_insert)

In [49]:
c.execute('INSERT INTO adoption_status (animal_id, adoption_status) SELECT animal_id, 0 FROM animal;')

In [50]:
sex_insert = '''INSERT INTO sex_type (sex_id, sex_desc) VALUES
('M', 'Male'),
('F', 'Female'),
('U', 'Unidentified'),
('N', 'Neutral'),
('S', 'Spayed');'''
c.execute(sex_insert)

In [51]:
c.execute("ALTER TABLE animal ADD COLUMN image BLOB")

In [52]:
for i in c.execute('select * FROM animal;'):
  print(i)

(1, 'OTHER', 'OSCAR', 'OWNER SUR', '2024-01-28', '6 YEARS', 'S', 'BROWN', 'TURTLE', 'M', 'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A451630&location=MONT', None, None)
(2, 'OTHER', 'FRANKLIN', 'OWNER SUR', '2024-01-28', '10 YEARS', 'M', 'RED', 'TURTLE / REDEARED SLIDER', 'M', 'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A439979&location=MONT', None, None)
(3, 'DOG', '*RITA', 'OWNER SUR', '2024-02-05', '2 YEARS', 'M', 'BRINDLE', 'AM PIT BULL TER', 'S', 'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A507522&location=MONT', None, None)
(4, 'OTHER', 'YERTLE', 'OWNER SUR', '2024-01-28', '7 YEARS', 'S', 'GREEN', 'TURTLE', 'F', 'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A444157&location=MONT', None, None)
(5, 'DOG', 'NOVA', 'EUTH REQ', '2023-12-08', '2 YEARS', 'M', 'TAN / WHITE', 'AM PIT BULL TER / MIX', 'S', 'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A513554&location=MONT', None, None)
(6, 'OTHER', 'SHELLDON', 'OWNER SUR', '2024-01-28', '10 YE

In [53]:
for i in c.execute('select intake_type_desc FROM intake;'):
  print(i)
conn.commit()
conn.close()

('Owner Surrender',)
('Euthanasia Request',)
('Stray',)
('Foster',)
('Boarding',)
('Return',)
